In [ ]:
import os
import subprocess
import re
import pandas as pd

encoding="cp932"
def searchFile(ROOT_PATH,saveFile):
    if not os.path.isdir(os.path.dirname(saveFile)):
        os.makedirs(os.path.dirname(saveFile))
    allFile={}
    count=0
    for pathname,dirnames,filenames in os.walk(ROOT_PATH):
        for filename in filenames:
            if filename.endswith(".java"):
                fullPath=os.path.join(pathname,filename)
                count=count+1
                if "package-info" not in fullPath:
                    parentFolder=re.sub(r"\\src\\.*","",fullPath)
                    if parentFolder in allFile:
                        allFile[parentFolder].append(fullPath)
                    else:
                        allFile.setdefault(parentFolder,[fullPath])
                        
    with open(saveFile,"w",encoding="utf-8") as f:              
        for key,item in allFile.items():
            for pathfile in item:
                print(pathfile,file=f)
            print(file=f)
    
def editClass(Spoonfile, CKfile, outPutFile):   
    df = pd.read_csv(CKfile,encoding=encoding)
    df = df[["file", "class","type", "dit", "fanin", "fanout","lcc","lcom*","loc","noc","rfc","innerClassesQty","totalFieldsQty","totalMethodsQty","wmc"]]
    CK=df.copy()
    CK["class"] = CK["class"].str.replace("$Anonymous", "$")
    spoon = pd.read_csv(Spoonfile,encoding=encoding)
    join = spoon.merge(CK, how="inner", on=["file", "class"])
    join["class"]=join["class"].str.replace("$",".",)
    join.to_csv(
        outPutFile,mode="w",index=False,encoding=encoding
    )

def editMethod(SpoonFile, CKFile, outPutFile):
    df = pd.read_csv(CKFile,encoding=encoding)
    df = df[["file", "class", "method","fanin","fanout", "wmc", "loc","returnsQty","variablesQty","parametersQty","loopQty","anonymousClassesQty","innerClassesQty","line"]]
    CK=df.copy()
    CK["class"] = CK["class"].str.replace("$Anonymous", "$")    
    spoon = pd.read_csv(SpoonFile,encoding=encoding)
    spoon["method"]=spoon["method"].str.replace("$",".")
    spoon["method"]=spoon["method"].str.replace(" ","")
    join = spoon.merge(CK, how="inner", on=["file", "class", "method"])
    join["class"]=join["class"].str.replace("$",".")
    join.to_csv(
        outPutFile,mode="w",index=False,encoding=encoding
    )
  
def editFile(saveMetricsFile,outPutFile):
    CKClass=saveMetricsFile+"class.csv"
    CKMethod=saveMetricsFile+"method.csv"
    SpoonClass=saveMetricsFile+"spoonClass.csv"
    SpoonMethod=saveMetricsFile+"spoonMethod.csv"
    saveClass=outPutFile+"joinClass.csv"
    saveMethod=outPutFile+"joinMethod.csv"
    
    editClass(SpoonClass,CKClass,saveClass)
    editMethod(SpoonMethod,CKMethod,saveMethod)

searchFile(os.path.join("C:\\Users\\sugii syuji","gitProject\\vert.x"),os.path.join("C:\\Users\\sugii syuji","machineLarnigData\\full.txt"))

In [103]:
editMethod("data\\corenlp\\1spoonMethod.csv","data\\corenlp\\1method.csv","data\\corenlp\\modifyMethod.csv")

In [104]:
df=pd.read_csv("data\\corenlp\\modifyMethod.csv")
print(len(df))

27579


In [ ]:
import pandas as pd
import shutil

def getCodeSmellClass():
    dfGod=pd.read_csv("ground truth\\GodClass.csv")
    dfRefused=pd.read_csv("ground truth\\RefusedBequest.csv")
    dfGod=dfGod[["nomeSistema","class"]]
    dfRefused=dfRefused[["nomeSistema","class"]]
    allClass=pd.concat([dfGod,dfRefused])
    allClass=allClass.drop_duplicates()
    return allClass
   
def createFolder(path):
    if os.path.isdir(path):
        shutil.rmtree(path)
    os.makedirs(path)

RootFolder="C:\\Users\\sugii syuji"
with open("tmp.txt","r",encoding="utf-8") as f:
    string=f.read() 
Projects=string.split("\n")

for block in Projects:
    blockFactor=block.split(",")
    smellFileName=blockFactor[3]
    project=os.path.join(RootFolder,"gitProject",blockFactor[0])      
    checkoutVersions=[blockFactor[1],blockFactor[2]]                                        

    saveFolder=os.path.join(os.getcwd(),"data",os.path.basename(project))
    saveTextFile=os.path.join(saveFolder,"full.txt")
    CKAnalyze=["java","-jar","ck-0.7.1-SNAPSHOT-jar-with-dependencies.jar",project,"False","0","True",saveFolder]
    SpoonAnalyze=["java","-jar","demo-1.0-snapshot.jar",saveTextFile,saveFolder]

    createFolder(saveFolder)

    allClass=getCodeSmellClass()
    group=allClass.groupby("nomeSistema")
    classes=group.get_group(smellFileName)

    i=1
    for version in checkoutVersions:
        gitCheckout=["git","checkout",version]
        subprocess.run(gitCheckout,cwd=project,check=True)
        searchFile(project,saveTextFile)
        SaveFile=os.path.join(saveFolder,str(i))
        CKAnalyze[7]=SaveFile
        SpoonAnalyze[4]=SaveFile
        subprocess.run(CKAnalyze,cwd=os.path.join(RootFolder,"ck\\target"),check=True)
        subprocess.run(SpoonAnalyze,cwd=os.path.join(os.getcwd(),"spoon\\target"),check=True)
        outPutFile=os.path.join(saveFolder,str(i))
        editFile(SaveFile,outPutFile)
        if i==1:
            print(smellFileName)
            df=pd.read_csv(outPutFile+"joinClass.csv",encoding=encoding)
            print(len(set(df['class']) & set(classes['class'])),"/",len(classes))
            data=(set(df['class']) & set(classes['class']))
            print(classes[~classes['class'].isin(df['class'])])

        i=i+1

In [127]:
def getCodeSmellClass():
    dfGod=pd.read_csv("ground truth\\GodClass.csv")
    dfRefused=pd.read_csv("ground truth\\RefusedBequest.csv")
    dfGod=dfGod[["nomeSistema","class"]]
    dfRefused=dfRefused[["nomeSistema","class"]]
    allClass=pd.concat([dfGod,dfRefused])
    allClass=allClass.drop_duplicates()
    return allClass
smellFileName="vertx"
allClass=getCodeSmellClass()
group=allClass.groupby("nomeSistema")
classes=group.get_group(smellFileName)
print(smellFileName)
df=pd.read_csv("data\\vert.x\\"+"1joinClass.csv",encoding=encoding)
print(len(set(df['class']) & set(classes['class'])),"/",len(classes))
data=(set(df['class']) & set(classes['class']))
print(classes[~classes['class'].isin(df['class'])])

vertx
76 / 76
Empty DataFrame
Columns: [nomeSistema, class]
Index: []


問題点：クラス数，メソッド数が微妙に違う

        ファイル名が異なるものがある

        ツールで計算する時，エラー・終了しない(checkstyle,jitwatch)

        ファイル名が長く保存できない(shenyu)

        ckツールと自作のプログラムで計測したものを結合する際，命名方法の違いで少なくなってしまう

In [71]:
dataFile=["analysis-ik","CoreNLP","dbeaver","fastjson","guava","HikariCP","java-faker","jedis","jenkins","jsoup","junit4","libgdx","mockserver","mybatis-3","nanohttpd","netty-socketio","redisson","retrofit","shopizer","spark","webmagic","xdm","ysoserial","zookeeper"]
for folder in dataFile:
    df1=pd.read_csv("data\\"+folder+"\\1joinClass.csv")
    df2=pd.read_csv("data\\"+folder+"\\1spoonMethod.csv",encoding="cp932")
    print(folder)
    print("class : ",len(df1))
    print("method : ",len(df2))
    print()

analysis-ik
class :  28
method :  210

CoreNLP
class :  4091
method :  35072

dbeaver
class :  6388
method :  38744

fastjson
class :  6147
method :  24654

guava
class :  5942
method :  62146

HikariCP
class :  159
method :  1385

java-faker
class :  197
method :  1326

jedis
class :  910
method :  7959

jenkins
class :  3780
method :  19941

jsoup
class :  325
method :  2710

junit4
class :  1419
method :  5274

libgdx
class :  3727
method :  47261

mockserver
class :  993
method :  7790

mybatis-3
class :  1497
method :  8288

nanohttpd
class :  140
method :  768

netty-socketio
class :  150
method :  786

redisson
class :  1971
method :  22537

retrofit
class :  272
method :  1090

shopizer
class :  1317
method :  8871

spark
class :  221
method :  1492

webmagic
class :  299
method :  1281

xdm
class :  303
method :  2046

ysoserial
class :  101
method :  378

zookeeper
class :  1569
method :  9464



In [ ]:
import pandas as pd
import shutil

def getCodeSmellClass():
    dfGod=pd.read_csv("ground truth\\GodClass.csv")
    dfRefused=pd.read_csv("ground truth\\RefusedBequest.csv")
    dfGod=dfGod[["nomeSistema","class"]]
    dfRefused=dfRefused[["nomeSistema","class"]]
    allClass=pd.concat([dfGod,dfRefused])
    allClass=allClass.drop_duplicates()
    return allClass
   
def createFolder(path):
    if os.path.isdir(path):
        shutil.rmtree(path)
    os.makedirs(path)

def doAmalyze():
    with open("tmp.txt","r",encoding="utf-8")as f:
        tmpString=f.read()
    Blocks=tmpString.split("\n")
    for Block in Blocks:
        ListFactor=Block.split(",")
        RootFolder="C:\\Users\\syuuj"
        smellFileName=ListFactor[2]
        project=os.path.join(RootFolder,"gitProject",ListFactor[0])      
        version=ListFactor[1]
        saveFolder=os.path.join(os.getcwd(),os.path.basename(project))
        saveTextFile=os.path.join(saveFolder,"full.txt")
        CKAnalyze=["java","-jar","ck-0.7.1-SNAPSHOT-jar-with-dependencies.jar",project,"False","0","True",saveFolder]
        SpoonAnalyze=["java","-jar","demo-1.0-snapshot.jar",saveTextFile,saveFolder]

        createFolder(saveFolder)

        allClass=getCodeSmellClass()
        group=allClass.groupby("nomeSistema")
        classes=group.get_group(smellFileName)

        i=1
        gitCheckout=["git","checkout",version]
        subprocess.run(gitCheckout,cwd=project,check=True)
        searchFile(project,saveTextFile)
        SaveFile=os.path.join(saveFolder,str(i))
        CKAnalyze[7]=SaveFile
        SpoonAnalyze[4]=SaveFile
        subprocess.run(CKAnalyze,cwd=os.path.join(RootFolder,"ck\\target"),check=True)
        subprocess.run(SpoonAnalyze,cwd=os.path.join(os.getcwd(),"spoon\\target"),check=True)
        outPutFile=os.path.join(saveFolder,str(i))
        editFile(SaveFile,outPutFile)
        if i==1:
            print(smellFileName)
            df=pd.read_csv(outPutFile+"joinClass.csv",encoding=encoding)
            print(len(set(df['class']) & set(classes['class'])),"/",len(classes))
            data=(set(df['class']) & set(classes['class']))
            print(classes[~classes['class'].isin(df['class'])])
        i=i+1
    
doAmalyze()

In [23]:
import pandas as pd
import os
import subprocess

RootFolder="C:\\Users\\sugii syuji"
def spoonAnalyze(textFile,saveFolder):
    SpoonAnalyze=["java","-jar","demo-1.0-snapshot.jar",textFile,saveFolder]
    subprocess.run(SpoonAnalyze,cwd=os.path.join(os.getcwd(),"spoon\\target"),check=True)

def ckAnalyze(gitProject,saveFolder):
    CKAnalyze=["java","-jar","ck-0.7.1-SNAPSHOT-jar-with-dependencies.jar",gitProject,"False","0","True",saveFolder]
    subprocess.run(CKAnalyze,cwd=os.path.join(RootFolder,"ck\\target"),check=True)
    
def gitCheckout(gitProject,version):
    checkout=["git","checkout",version]
    subprocess.run(checkout,cwd=gitProject,check=True)
    
gitProject=os.path.join(RootFolder,"gitProject","vert.x")
saveFolder=os.path.join(os.getcwd(),"vert.x\\2-2")
textFile=os.path.join(os.getcwd(),"full.txt")
spoonAnalyze(textFile,saveFolder)
ckAnalyze(gitProject,saveFolder)
editFile(saveFolder,saveFolder)


In [64]:
project="vert.x"
i=1
while i<=2:
    file=project+"\\"+str(i)
    df1=pd.read_csv(file+"-1joinClass.csv")
    df2=pd.read_csv(file+"-2joinClass.csv")
    join=pd.concat([df1,df2])
    drop=join.drop_duplicates()
    drop.to_csv(file+"joinClass.csv")
    
    df3=pd.read_csv(file+"-1joinMethod.csv")
    df4=pd.read_csv(file+"-2joinMethod.csv")
    join2=pd.concat([df3,df4])
    drop2=join2.drop_duplicates()
    drop2.to_csv(file+"joinMethod.csv")
    i=i+1

In [ ]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
df1=pd.read_csv("data\\corenlp\\1spoonMethod.csv")
df2=pd.read_csv("data\\corenlp\\1method.csv")
df1["flag"]=1
df2["flag"]=2
df1=df1[["file","class","method","line","flag"]]
df2=df2[["file","class","method","line","flag"]]
df2["class"]=df2["class"].str.replace("$Anonymous","$")
df1["method"]=df1["method"].str.replace("$",".")
df1["method"]=df1["method"].str.replace(" ","")
df2["method"]=df2["method"].str.replace(r"<[^>]*>", "", regex=True)
join=pd.concat([df1,df2])
drop=join.drop_duplicates(keep=False,subset=["file","class","method"])
drop2=drop.drop_duplicates(keep=False,subset=["file","class","line"])
drop3=drop2
drop3.loc[drop3["flag"] == 1, "line"] += 1
drop3=drop3.drop_duplicates(keep=False,subset=["file","class","line"])
print(len(df1),len(df2),len(join),len(drop))
print(len(drop2))
print(len(drop3))
print(drop3)

#65992 32991

35072 33717 68789 8889
3371
2797
                                                    file  \
1      C:\Users\syuuj\gitProject\CoreNLP\examples\sam...   
3      C:\Users\syuuj\gitProject\CoreNLP\examples\sam...   
5      C:\Users\syuuj\gitProject\CoreNLP\examples\sam...   
8      C:\Users\syuuj\gitProject\CoreNLP\itest\src\ed...   
12     C:\Users\syuuj\gitProject\CoreNLP\itest\src\ed...   
20     C:\Users\syuuj\gitProject\CoreNLP\itest\src\ed...   
24     C:\Users\syuuj\gitProject\CoreNLP\itest\src\ed...   
27     C:\Users\syuuj\gitProject\CoreNLP\itest\src\ed...   
44     C:\Users\syuuj\gitProject\CoreNLP\itest\src\ed...   
47     C:\Users\syuuj\gitProject\CoreNLP\itest\src\ed...   
62     C:\Users\syuuj\gitProject\CoreNLP\itest\src\ed...   
64     C:\Users\syuuj\gitProject\CoreNLP\itest\src\ed...   
69     C:\Users\syuuj\gitProject\CoreNLP\itest\src\ed...   
79     C:\Users\syuuj\gitProject\CoreNLP\itest\src\ed...   
83     C:\Users\syuuj\gitProject\CoreNLP\itest\src\ed...   
88     